In [2]:
## 데이터 수집
# 크롤링을 이용한 서울시 스타벅스 매장 목록 데이터 생성

from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd

In [3]:
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
url = 'https://www.starbucks.co.kr/store/store_map.do?disp=locale'
driver.get(url)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 105.0.5195
[WDM] - Get LATEST chromedriver version for 105.0.5195 google-chrome
[WDM] - About to download new driver from https://chromedriver.storage.googleapis.com/105.0.5195.52/chromedriver_win32.zip
[WDM] - Driver has been saved in cache [C:\Users\keb20\.wdm\drivers\chromedriver\win32\105.0.5195.52]


In [5]:
# '서울' 클릭
seoul_btn = '#container > div > form > fieldset > div > section > article.find_store_cont > article > article:nth-child(4) > div.loca_step1 > div.loca_step1_cont > ul > li:nth-child(1) > a'
driver.find_element(By.CSS_SELECTOR, seoul_btn).click()

In [6]:
# '전체' 클릭
all_btn = '#mCSB_2_container > ul > li:nth-child(1) > a'
driver.find_element(By.CSS_SELECTOR, all_btn).click()

In [7]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [8]:
starbucks_soup_list = soup.select('li.quickResultLstCon')
print(len(starbucks_soup_list))

591


In [9]:
starbucks_soup_list[0]

<li class="quickResultLstCon" data-code="3762" data-hlytag="null" data-index="0" data-lat="37.501087" data-long="127.043069" data-name="역삼아레나빌딩" data-storecd="1509" style="background:#fff"> <strong data-my_siren_order_store_yn="N" data-name="역삼아레나빌딩" data-store="1509" data-yn="N">역삼아레나빌딩  </strong> <p class="result_details">서울특별시 강남구 언주로 425 (역삼동)<br/>1522-3232</p> <i class="pin_general">리저브 매장 2번</i></li>

In [12]:
starbucks_store = starbucks_soup_list[0]
name = starbucks_store.select('strong')[0].text.strip() ## 문자열에서 특정문자 제거(parameter가 없으므로 공백 제거)
lat = starbucks_store['data-lat'].strip()
lng = starbucks_store['data-long'].strip()
store_type = starbucks_store.select('i')[0]['class'][0][4:] # 0번째 i 태그의 0번째 class에서 4번째 글자부터
address = str(starbucks_store.select('p.result_details')[0]).split('<br/>')[0].split('>')[1]
tel = str(starbucks_store.select('p.result_details')[0]).split('<br/>')[1].split('<')[0]

print(name)
print(lat)
print(lng)
print(store_type)
print(address)
print(tel)

역삼아레나빌딩
37.501087
127.043069
general
서울특별시 강남구 언주로 425 (역삼동)
1522-3232


In [13]:
starbucks_list = []
for item in starbucks_soup_list:
    name = item.select('strong')[0].text.strip()
    lat = item['data-lat'].strip()
    lng = item['data-long'].strip()
    store_type = item.select('i')[0]['class'][0][4:]
    address = str(item.select('p.result_details')[0]).split('<br/>')[0].split('>')[1]
    tel = str(item.select('p.result_details')[0]).split('<br/>')[1].split('<')[0]
    
    starbucks_list.append([name, lat, lng, store_type, address, tel])

In [14]:
columns = ['매장명', '위도', '경도', '매장타입', '주소', '전화번호']
seoul_starbucks_df = pd.DataFrame(starbucks_list, columns = columns)
seoul_starbucks_df.head()

,매장명,위도,경도,매장타입,주소,전화번호
0,역삼아레나빌딩,37.501087,127.043069,general,서울특별시 강남구 언주로 425 (역삼동),1522-3232
1,논현역사거리,37.510178,127.022223,general,서울특별시 강남구 강남대로 538 (논현동),1522-3232
2,신사역성일빌딩,37.5139309,127.0206057,general,서울특별시 강남구 강남대로 584 (논현동),1522-3232
3,국기원사거리,37.499517,127.031495,general,서울특별시 강남구 테헤란로 125 (역삼동),1522-3232
4,대치재경빌딩R,37.494668,127.062583,reserve,서울특별시 강남구 남부순환로 2947 (대치동),1522-3232


In [15]:
seoul_starbucks_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 591 entries, 0 to 590
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   매장명     591 non-null    object
 1   위도      591 non-null    object
 2   경도      591 non-null    object
 3   매장타입    591 non-null    object
 4   주소      591 non-null    object
 5   전화번호    591 non-null    object
dtypes: object(6)
memory usage: 27.8+ KB


In [16]:
seoul_starbucks_df.to_excel('../2022 하계 모각코 자료/6_Starbucks_Location/files/1_seoul_starbucks_list.xlsx', index = False)